# **Context aware chatbot.**

Chat bot for informative answers based on the conversation history and retrieved documents. It utilizes ChatOpenAI from the langchain_openai library and implements a retrieval system using FAISS. In this case, the relevant document is a JSON file.
Using ChatPromptTemplate, the code generates responses based on retrieved documents, conversation history, and the user's input.


References:

https://github.com/BelgiAmir/TenMinutesChatbot/blob/main/LangChain%20Demo.ipynb

https://streamlit.io/

https://www.langchain.com/

https://medium.com/@ahmed.mohiuddin.architecture/using-ai-to-chat-with-your-documents-leveraging-langchain-faiss-and-openai-3281acfcc4e9

## **Installation & Imports**

In [ ]:
%pip install langchain
%pip install langchain-openai
%pip install jq
%pip install faiss-cpu
%pip install streamlit

In [ ]:
import streamlit as st

from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

import json
from pathlib import Path
from pprint import pprint

## **Build the app**


### Title and side bar

In [ ]:
st.title("💬 Laboratory company Chatbot"")
st.caption("Chatbot that answers questions about UV detector catalog")

# Sidebar for API key input
with st.sidebar:
    OPENAI_API_KEY = st.text_input("OpenAI API Key", key="chatbot_api_key", type="password")
    "[Get an OpenAI API key](https://platform.openai.com/account/api-keys)"
    "[View the source code](https://github.com/franconti/context_aware_chatbot)"

# Check for API key
if not OPENAI_API_KEY:
    st.info("Please add your OpenAI API key to continue.")
    st.stop()

### Create the llm API wrapper

In [ ]:
llm = ChatOpenAI(openai_api_key= OPENAI_API_KEY, model_name="gpt-3.5-turbo")

### Implementing Chatbot With Memory

In [ ]:
# Load and process the data
loader = JSONLoader(
    file_path='UVdetectors.json',
    jq_schema='.UVDetectors.models[]',
    text_content=False)
docs = loader.load()

# Split the data
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

# Embedd the data (to convert text into numerical vector representations)
embeddings = OpenAIEmbeddings(openai_api_key= OPENAI_API_KEY)
vector = FAISS.from_documents(documents, embeddings)

# Create retriever to query the vector database
retriever = vector.as_retriever()

### Creating the retrival chain - Giving the chain ability to rereive data

In [ ]:
# Create the retrieval chain
"""Combines the language model (llm), the retriever (which is a FAISS vector), and the prompt template to
 create a chain that is capable of generating search queries based on the conversation history and retrieving relevant documents."""

retrieval_chain_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, retrieval_chain_prompt)

### Creating the documents chain - Answering the user input

In [ ]:
# Create the document chain that guides the chatbot on how to interact with users.
document_chain_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "This is a chatbot for BiologyArte, a laboratory supplies company.\
      The chatbot's goal is to provide information and assistance to potential and existing BiologyArte customers.. \
      The chatbot should decline to answer any question not related to the company goal. \
      The chatbot should be friendly, polite, and helpful. \
      The chatbot should refer the user to the official website or a human representative if it cannot answer the question or handle the request. \
      Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, document_chain_prompt)

### Combining both chains

In [ ]:
combined_chain = create_retrieval_chain(retriever_chain, document_chain)

### Initialize chat history

In [ ]:
if "messages" not in st.session_state:
    st.session_state["messages"] = [{"role": "assistant", "content": "How can I help you?"}]


### Display chat messages

In [ ]:
for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

### Handle user input

In [ ]:
if prompt := st.chat_input():
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)
    response = combined_chain.invoke({"input": prompt, "chat_history": st.session_state.messages})
    msg = response["answer"]
    st.session_state.messages.append({"role": "assistant", "content": msg})
    st.chat_message("assistant").write(msg)
